In [ ]:
# -*- coding: utf-8 -*-
# install libraries
%pip install --upgrade tiktoken
%pip install --upgrade openai

In [1]:
# import libraries
import os
import time
import tiktoken
import concurrent.futures
from openai import AzureOpenAI

# Funções que serão utilizadas durante os exemplos

In [ ]:

# Nome dos modelos Azure OpenAI criados na subscrição 
GPT_35_TURBO = "gpt-35-turbo"
GPT_4 = "gpt-4"
GPT_4_32K = "gpt-4-32k"
GPT_4_TURBO = "gpt-4-turbo"
TEXT_EMBEDDING_ADA_002 = "text-embedding-ada-002"

# carregar tokenizador para os modelos de linguagem
encoding = tiktoken.get_encoding("cl100k_base")

# inicialização do cliente Azure OpenAI
client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2024-02-15-preview"
)

# função para chamar o modelo de linguagem
def call_llm(deployment_name, system_message, question):
    start_time = time.perf_counter()
    response = client.chat.completions.create(
        model = deployment_name,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": question},
        ]
    )
    elapsed_time = time.perf_counter() - start_time
    resp = response.choices[0].message.content
    tokens_completion = num_tokens_from_string(resp, deployment_name)
    tokens_prompt = num_tokens_from_string(system_message, deployment_name) + num_tokens_from_string(question, deployment_name)
    return {"model": deployment_name, "elapsed_time": elapsed_time, 
            "num_tokens_completion": tokens_completion, 
            "tokens_prompt": tokens_prompt, 
            "response": resp, 
            "question": question, 
            "system_message": system_message}

# funcao para contar tokens
def num_tokens_from_string(texto, model):
    encoding = tiktoken.encoding_for_model(model)
    num_tokens = len(encoding.encode(texto))
    return num_tokens

# função para gerar embeddings
def generate_embeddings(text, model):
    start_time = time.perf_counter()
    embeddings = client.embeddings.create(input = [text], model=model).data[0].embedding
    elapsed_time = time.perf_counter() - start_time
    return embeddings, elapsed_time

# função para executar modelos simultaneamente
# models: lista de modelos
# system_message: mensagem do sistema
# question: pergunta
# retorna uma lista com os resultados
def execute_simultaneously(models, system_message, question):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(call_llm, model, system_message, question) for model in models]
        results = []
        for future in concurrent.futures.as_completed(futures):
            print(future.result())
            results.append(future.result())
    return results


In [ ]:
def sample_agenda():
    agenda = '''12/03/2022 12:00:00 Ocupado
        12/03/2022 13:00:00 Ocupado
        12/03/2022 14:00:00 Ocupado
        12/03/2022 15:00:00 Livre
        12/03/2022 16:00:00 Ocupado
        12/03/2022 17:00:00 Livre
        12/03/2022 18:00:00 Ocupado
        12/03/2022 19:00:00 Livre
        12/03/2022 20:00:00 Ocupado'''
    return call_llm(GPT_4, "Você é um bot que retorna as respostas em formato json", f"Com base nessa agenda, qual o próximo horário livre? {agenda}")

sample_agenda()

In [ ]:
def sample_system_message_intonation():
    system_message = {"despojado":"Você é jum assitente que sempre responde em tom despojado, de forma didática e em porguguês, apesar de eventualmente citar palavras em inglês.",
                      "formal":"Você é um assistente que sempre responde em tom formal, de forma didática e em português, sem citar palavras em inglês.",
                      "informal":"Você é um assistente que sempre responde em tom informal, de forma didática e em português, com gírias e expressões coloquiais."}
    question = "O que é inteligência artificial?"

    for item in system_message:
        print(item)
        print(call_llm(GPT_35_TURBO, system_message[item], "O que é inteligência artificial?"))

sample_system_message_intonation()

# Exemplo de diferença entre modelos

In [ ]:
system_message = "Você é um bot que sempre retorna as respostas em formato json e em português."
question = "Conte-me um paradoxo de maneira estrudurada e sucinta com o mínimo de detalhes possível."
models = [GPT_35_TURBO, GPT_4, GPT_4_32K, GPT_4_TURBO]
results = execute_simultaneously(models, system_message, question)

# Exemplo de Classificação de texto

In [ ]:
normativo_classificado = """
"""

normativo = ''''''

system_message = "Você é um assistente capaz de compreender estrutura de textos da língua portuguesa (Brasil), sendo capaz de identificar parágrafos, títulos, estrofes, versos e frases. Especialmente, texto em formato de normativos (leis, decretos, portarias, resoluções, etc)."
question = f"Com base na classificação de parágrafos desse normartivo: [{normativo_classificado}], responda a pergunta: quantos paragráfos o seguinte normativo possui: [{normativo}]?."

call_llm(GPT_4_32K, system_message, question)